In [1]:
# Set up imports
import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler as lr_scheduler
from torchvision import datasets, transforms
import model_flower
import model_train
import model_test
import os
import datetime

# Set up variables
tr_batchsize = 16 # The size of the training batches
val_test_batchsize = 16 # The size of the validation / testing batches
epochs = 500 # The number of epochs to do
validate_steps = 750 # The number of steps to complete before validation
learning_rate = 0.00005 # The learning rate to start at
load_model = False # If a model should be requested to be loaded, or not
save_model = True # If the model should be saved after testing, or not

# The actual model variables
model = None
criterion = None
optimizer = None
scheduler = None

# Model file values. If "None", then they haven't loaded successfully
md = None
lr = None
sch = None
cri = None
opt = None

In [2]:
# By default, set to use the CPU
deviceFlag = torch.device('cpu')

# If a GPU is available, use it
if torch.cuda.is_available():
    print(f'Found {torch.cuda.device_count()} GPUs.')
    deviceFlag = torch.device('cuda:0') # Default to cuda 0, but can be changed.

print(f'Now the deivce is set to {deviceFlag}')

Found 1 GPUs.
Now the deivce is set to cuda:0


# Data Loading and Transformations

In [3]:
training_transforms = transforms.Compose([
    # Randomly rotate it 90 degrees
    transforms.RandomRotation(90),
    # Randomly sharpen the image
    transforms.RandomAdjustSharpness(1.5, 0.5),
    # Randomly crop an area of the flower of size 224x224
    transforms.RandomResizedCrop(224),
    # Flip it horizontally, or don't
    transforms.RandomHorizontalFlip(),
    # Flip it vertically, or don't
    transforms.RandomVerticalFlip(),
    # Convert the image to a Tensor
    transforms.ToTensor(),
    # Normalize the Tensor values so that they're easier for the
    # model to train from
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

validation_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], # RGB mean & std estied on ImageNet
                         [0.229, 0.224, 0.225])
])

testing_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], # RGB mean & std estied on ImageNet
                         [0.229, 0.224, 0.225])
])

# Load the datasets of the Flower102 images
train_dataset = datasets.Flowers102(root = './dataset', split = 'train', transform = training_transforms, download = True)
valid_dataset = datasets.Flowers102(root = './dataset', split = 'val', transform = validation_transforms, download = True)
test_dataset = datasets.Flowers102(root = './dataset', split = 'test', transform = testing_transforms, download = True)


# Create the loaders for the datasets, to be used to train, validate and test the model
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = tr_batchsize,
                                           shuffle = True)

validate_loader = torch.utils.data.DataLoader(dataset = valid_dataset,
                                           batch_size = val_test_batchsize)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = val_test_batchsize)

In [4]:
loaded_file = False

file_name = "ERROR"
# If going to load a model
if load_model:
    # First request the file name of a model
    file_name = "models/" + input("Model to load: ")
    # And try to load it
    if os.path.exists(file_name):
        try:
            file_data = torch.load(file_name)
            md = file_data["model"]
            lr = file_data["learning_rate"]
            sch = file_data["scheduler"]
            cri = file_data["criterion"]
            opt = file_data["optimizer"]
            loaded_file = True
            print("Model loaded.")
        except:
            # If it fails, load nothing from the file
            md = None
            lr = learning_rate
            sch = None
            cri = None
            opt = None
            print("Model failed to load. Using default untrained Model.")

print("Creating Model...")
model = model_flower.FlowerModel()
print("Model created. Moving the Model to " + deviceFlag.type + "...")
model.to(deviceFlag)
print("Moved the Model to " + deviceFlag.type + ".")

if loaded_file:
    print("\nUsing model file from " + file_name)
    model.load_state_dict(md)
    learning_rate = lr

Creating Model...
Model created. Moving the Model to cuda...
Moved the Model to cuda.


# Define Loss Function and Optimizer

In [5]:
# Negative Log Likelihood Loss
# criterion = nn.NLLLoss()

# Cross Entropy Loss
criterion = nn.CrossEntropyLoss()
if not cri is None:
    criterion.load_state_dict(cri)

# optimizer 1
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
if not opt is None:
    optimizer.load_state_dict(opt)

# optimizer 2
# optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay = 0.005, momentum = 0.9)

In [6]:
# Scheduler
scheduler = lr_scheduler.StepLR(optimizer, 500, learning_rate)
if not sch is None:
    scheduler.load_state_dict(sch)

TypeError: 'NoneType' object is not iterable

# Train Model

In [ ]:
model_train.train_classifier(model, train_loader, validate_loader, optimizer, criterion,
                             optim_scheduler=scheduler, device_flag=deviceFlag, epochs=epochs,
                             validate_steps=validate_steps, validate_stepped=True, validate_epoch=False,
                             validate_end=True)

# Test Model

In [ ]:
model_test.test_accuracy(model, test_loader, device_flag=deviceFlag)

# Save Model

In [ ]:
if save_model:
    save_data = {
        "learning_rate": optimizer.param_groups[0]['lr'],
        "model": model.state_dict(),
        "criterion": criterion.state_dict(),
        "optimizer": optimizer.state_dict(),
        "scheduler": scheduler.state_dict()
    }

    torch.save(save_data, "models/" + str(datetime.datetime.now()).replace(":","-")
               + f" b{tr_batchsize}-e{epochs}-lr{learning_rate}" "-model.pt")

In [ ]:
# Stop Run All here
assert False

In [ ]:
# Reload imports in the case that they are changed
from importlib import reload

# If not loaded into cache yet, import them
import model_flower
import model_train
import model_test

reload(model_flower)
reload(model_train)
reload(model_test)

In [ ]:
save_model = True